In [10]:
%cd /netapp/l.lemikhova/projects/VOS_forked/vos

/netapp/l.lemikhova/projects/VOS_forked/vos


In [11]:
import argparse
import csv
import cv2
import json
import os
from detection.core.datasets import metadata
from matplotlib import pyplot as plt
from tqdm import tqdm
import imagesize

/netapp/l.lemikhova/anaconda3/envs/vos/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
def get_fruits_dicts(stage, root="./data/OpenIm", output_dir=None, dataset='openim'): 
    root_ann = f'{root_img}/annotation'
    root_img = f'{root}/images'
    openim_classes_need = metadata.OPENIM_FRUITS_ID_THING_CLASSES
    # openim_classes_need = ['banana', 'apple', 'orange', 'strawberry', 'tomato', 'lemon', 'pear', 'grape', 'watermelon', 'pineapple', 'pomegranate', 'grapefruit', 'peach', 'mango', 'common_fig', 'cantaloupe']
    category_mapper = dict(zip(openim_classes_need, range(len(openim_classes_need))))
    images_list_id = []
    annotations_list_id = []    
    images_list_ood = []
    annotations_list_ood = []   
    f = open(f'{root_ann}/{dataset}_fruits_all/fruits_{stage}.txt')
    filename = str.rstrip(f.readline()) # 'The first line.\n'
    idx = 0
    count = 0
    while filename != '':
        annotation_list_file_id = []
        full_filename = os.path.join(root_img, filename)
        if not os.path.exists(full_filename):
            num = int(f.readline())
            for _ in range(num):
                line = f.readline()
            filename = str.rstrip(f.readline()) # 'The first line.\n'
            continue
        width, height = imagesize.get(full_filename)
        num = int(f.readline())
        num_ID_boxes = 0
        has_ood = False
        for i in range(num):
            line = f.readline()
            bbox = list(map(float, line.split(' ')[:4]))
            category_name = ' '.join(line.split(' ')[4:])[:-1]
            if category_name not in category_mapper.keys():
                has_ood = True
                continue
            label = category_mapper[category_name]
            annotation_list_file_id.append({'image_id': idx,
                                    'id': count,
                                    'category_id': label,
                                    'bbox': bbox,
                                    'area': bbox[2] * bbox[3],
                                    'iscrowd': 0,
                                    'is_truncated': 0,
                                    'is_occluded': 0})
            count += 1
            num_ID_boxes += 1
        condition_id = num_ID_boxes > 0 and not has_ood
        condition_ood = num_ID_boxes == 0
        if condition_id:
            annotations_list_id += annotation_list_file_id
            images_list_id.append({'id': idx,
                            'width': width,
                            'height': height,
                            'file_name': filename,
                            'license': 1})
        elif condition_ood:
            images_list_ood.append({'id': idx,
                            'width': width,
                            'height': height,
                            'file_name': filename,
                            'license': 1})
        filename = str.rstrip(f.readline()) # 'The first line.\n'
        idx += 1
    for images_list, annotations_list, output_dir in zip([images_list_id, images_list_ood], [annotations_list_id, annotations_list_ood], [f'{root_ann}/{dataset}_id/COCO-Format', f'{root_ann}/{dataset}_ood/COCO-Format']):
        categories = [{"supercategory": "food", "id": i, "name": classname} for classname, i in category_mapper.items()]
        licenses = [{'id': 1,
                    'name': 'none',
                    'url': 'none'}]
        json_dict = {'info': {'year': 2020},
                    'licenses': licenses,
                    'categories': categories,
                    'images': images_list,
                    'annotations': annotations_list}
        os.makedirs(output_dir, exist_ok=True)
        file_name = f'{output_dir}/{stage}_coco_format.json'
        with open(file_name, 'w') as outfile:
            json.dump(json_dict, outfile)
    print('num id_bboxes:', count)
    print('len(images_list_id):', len(images_list_id))
    print('len(images_list_ood):', len(images_list_ood))
    return 0

In [19]:
# for stage in ['test', 'val', 'train']:
#     print(stage)
#     get_fruits_dicts(stage, root="./data/OpenIm", output_dir=None, dataset='openim')

In [21]:
for stage in ['test', 'val', 'train']:
    print(stage)
    get_fruits_dicts(stage=stage, root="./data/COCO", output_dir=None, dataset='coco')

test
num id_bboxes: 7983
len(images_list_id): 429
len(images_list_ood): 20
val
num id_bboxes: 9843
len(images_list_id): 389
len(images_list_ood): 10
train
num id_bboxes: 62076
len(images_list_id): 2218
len(images_list_ood): 70


## OOD similar and different

In [16]:
def get_sim_diff_dicts(stage, root_ann="./data/OpenIm/annotation", root_img="./data/OpenIm", dataset='openim'):  #'/content/coco'
    openim_classes_need = metadata.OPENIM_FRUITS_ID_THING_CLASSES
    # ood = 'grape', 'watermelon', 'pineapple', 'pomegranate', 'grapefruit', 'peach', 'mango', 'common fig', 'cantaloupe'
    ood_sim = ['grape', 'grapefruit', 'peach', 'mango']
    ood_diff = ['watermelon', 'pineapple', 'pomegranate','common fig', 'cantaloupe']
    category_mapper = dict(zip(openim_classes_need, range(len(openim_classes_need))))
    images_list_id = []
    images_list_ood = []
    images_list_ood_sim = []
    images_list_ood_diff = []
    annotations_list_id = []    
    annotations_list_ood = []   
    f = open(f'{root_ann}/openim_fruits_all/fruits_{stage}.txt')
    filename = str.rstrip(f.readline()) # 'The first line.\n'
    idx = 0
    count = 0
    while filename != '':
        annotation_list_file_id = []
        full_filename = os.path.join(root_img, filename)
        if not os.path.exists(full_filename):
            num = int(f.readline())
            for _ in range(num):
                line = f.readline()
            filename = str.rstrip(f.readline()) # 'The first line.\n'
            continue
        img = cv2.imread(full_filename)
        height, width = img.shape[:2]
        num = int(f.readline())
        num_ID_boxes = 0
        has_ood = False
        categories = []
        for i in range(num):
            line = f.readline()
            bbox = list(map(float, line.split(' ')[:4]))
            category_name = ' '.join(line.split(' ')[4:])[:-1]
            if category_name not in category_mapper.keys():
                has_ood = True
                categories.append(category_name)
                continue
            label = category_mapper[category_name]
            annotation_list_file_id.append({'image_id': idx,
                                    'id': count,
                                    'category_id': label,
                                    'bbox': bbox,
                                    'area': bbox[2] * bbox[3],
                                    'iscrowd': 0,
                                    'is_truncated': 0,
                                    'is_occluded': 0})
            count += 1
            num_ID_boxes += 1
        condition_id = num_ID_boxes > 0 and not has_ood
        condition_ood = num_ID_boxes == 0 and has_ood
        if condition_id:
            annotations_list_id += annotation_list_file_id
            images_list_id.append({'id': idx,
                            'width': width,
                            'height': height,
                            'file_name': filename,
                            'license': 1})
        elif condition_ood:
            images_list_ood.append({'id': idx,
                            'width': width,
                            'height': height,
                            'file_name': filename,
                            'license': 1})
            if min([category in ood_sim for category in categories]):
                images_list_ood_sim.append({'id': idx,
                                'width': width,
                                'height': height,
                                'file_name': filename,
                                'license': 1})
            elif min([category in ood_diff for category in categories]):
                images_list_ood_diff.append({'id': idx,
                                'width': width,
                                'height': height,
                                'file_name': filename,
                                'license': 1})
        filename = str.rstrip(f.readline()) # 'The first line.\n'
        idx += 1
    for images_list, annotations_list, output_dir in zip([images_list_ood_sim, images_list_ood_diff], [annotations_list_ood, annotations_list_ood], [f'{root_ann}/{dataset}_ood_sim/COCO-Format', f'{root_ann}/{dataset}_ood_diff/COCO-Format']):
        categories = [{"supercategory": "food", "id": i, "name": classname} for classname, i in category_mapper.items()]
        licenses = [{'id': 1,
                    'name': 'none',
                    'url': 'none'}]
        json_dict = {'info': {'year': 2020},
                    'licenses': licenses,
                    'categories': categories,
                    'images': images_list,
                    'annotations': annotations_list}
        os.makedirs(output_dir, exist_ok=True)
        file_name = f'{output_dir}/{stage}_coco_format.json'
        with open(file_name, 'w') as outfile:
            json.dump(json_dict, outfile)
    print(count)
    print('images_list_id', len(images_list_id))
    print('images_list_ood', len(images_list_ood))
    print('images_list_ood_sim', len(images_list_ood_sim))
    print('images_list_ood_diff', len(images_list_ood_diff))
    return 0

In [17]:
for stage in ['test', 'val', 'train']:
    print(stage)
    get_sim_diff_dicts(stage)


test
906
images_list_id 209
images_list_ood 85
images_list_ood_sim 30
images_list_ood_diff 50
val
2724
images_list_id 617
images_list_ood 153
images_list_ood_sim 29
images_list_ood_diff 116
train
16779
images_list_id 3500
images_list_ood 873
images_list_ood_sim 202
images_list_ood_diff 610


## COCO neg

In [18]:
get_fruits_dicts(stage='test', root_ann="./data/coco_neg", root_img="/ssd/l.lemikhova/data/animals/coco/full/val2017", output_dir=None, dataset='coco_neg')

0
0
500


0

## Deep fruits

In [4]:
get_fruits_dicts('test', root_ann="./data/deep_fruits", root_img="./data/deep_fruits", dataset='deep_fruits')

981
163
166


0

### Check that no id in ood

In [ ]:
stage = 'test'
root = "./data/deep_fruits"
openim_classes_need = metadata.OPENIM_FRUITS_ID_THING_CLASSES
category_mapper = dict(zip(openim_classes_need, range(len(openim_classes_need))))
images_list_id = []
annotations_list_id = []    
images_list_ood = []
annotations_list_ood = []   
f = open(f'{root}/annotation/fruits_{stage}.txt')
filename = str.rstrip(f.readline()) # 'The first line.\n'
idx = 0
count = 0
ood_num = 0
while filename != '' and ood_num < 10:
    annotation_list_file_id = []
    full_filename = os.path.join(root, filename)
    img = cv2.imread(full_filename)
    height, width = img.shape[:2]
    num = int(f.readline())
    num_ID_boxes = 0
    categories = []
    has_ood = False
    for i in range(num):
        line = f.readline()
        bbox = list(map(float, line.split(' ')[:4]))
        category_name = ' '.join(line.split(' ')[4:])[:-1]
        if category_name not in category_mapper.keys():
            has_ood = True
            categories.append(category_name)
            continue
        label = category_mapper[category_name]
        categories.append(category_name)
        count += 1
        num_ID_boxes += 1
    condition_id = num_ID_boxes > 0 and not has_ood
    condition_ood = num_ID_boxes == 0 and has_ood
    # if condition_id:
    #     annotations_list_id += annotation_list_file_id
    #     images_list_id.append({'id': idx,
    #                     'width': width,
    #                     'height': height,
    #                     'file_name': filename,
    #                     'license': 1})
    if condition_ood:
        print(categories)
        ood_num += 1
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.show()
    filename = str.rstrip(f.readline()) # 'The first line.\n'
    idx += 1